In [5]:
%matplotlib inline
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt #繪圖用
from pyquaternion import Quaternion #用於旋轉

## 版本更新
1.修改函數名稱
2.刪除舊版函數
3.新torsion

# PDB轉成Dataframe

In [6]:
def pdb_to_df(pdb): 
    with open(pdb,'r',) as f: #開啟檔案
        lines = []
        for line in f.readlines() :
            line = line.strip() #去除/n
            line = line.split() #以空白分割
            lines.append(line) 
    df = pd.DataFrame(lines,columns = list("abcdefghijk"),dtype=np.float64) 
    # 轉成dataframe並用abcd代替上面columns，方便取出需要之編號
    df = df.loc[:, ("c","e","g","h","i")]
    df.columns = ["element","chain", "X", "Y", "Z"]
    #df.index = range(1,len(df)+1) #index改成1開始
    mask = df["element"].isin(["C","N","CA"]) #只抓取胺基酸主鍊N,CA,C
    df = df[mask] 
    df["backbone_index"] = df.index #保留原始骨架的index
    return df 

# 向量長度
向量(x, y, z)之長度為 $√(x^2+y^2+z^2)$

用於對dataframe之運算

In [7]:
def df_longth(v):
    return np.linalg.norm(v,axis=1)

用於小處之運算

In [8]:
def longth(v):
    return np.sqrt(np.sum(np.dot(v,v))) 

# 兩向量計算夾角

二維矩陣用之dot

In [9]:
def df_dot(v1, v2): 
    return np.sum(np.multiply(v1, v2), axis=1, dtype=np.float64)

因為multply只支援同大小的array故把它變成三倍,m2.T是轉置矩陣

In [10]:
def tothree(m):
    m2 = np.array([m, m, m]) 
    return m2.T # .T表示該矩陣之轉置矩陣

### 角度
Cos = dot(v1, v2)/longth(v1)*longth(v2)
但左右之cos值相同因此無法算出正負角度

In [11]:
def angle(v1, v2, l1="check", l2="check"):
    if l1 == "check" and l2 == "check": #若l1,l2沒有值再進行運算，提高速度
        cos = df_dot(v1,v2)/(df_longth(v1)*df_longth(v2)) 
    else:
        cos = df_dot(v1,v2)/(l1*l2) 
    return np.arccos(cos)

### 向量
後項XYZ-前項XYZ

In [12]:
def df_vecter(df):
    df[["VX", "VY", "VZ"]] = df.loc[:, ("X", "Y", "Z")].diff()

### 鍵長 
$√(Vx^2+Vy^2+Vz^2)$

In [13]:
def df_bond_longth(df):
    df["bond_longth"] = df_longth(df.loc[:, ("VX", "VY", "VZ")].values)

### 鍵角
$(V1·V2)/(l1*l2)$

In [14]:
def df_bond_angle(df):
    df["bond_angle"] = np.NAN
    vxyz = ("VX", "VY", "VZ")
    v1 = df.loc[1:len(df)-1, vxyz].values #1-df-1
    v2 = df.loc[2:len(df), vxyz].values
    l1 = df.loc[1:len(df)-1, "bond_longth"].values
    l2 = df.loc[2:len(df), "bond_longth"].values
    df.loc[2:len(df), "bond_angle"] = angle(-v1, v2, l1, l2)

### torsion
假設有四點a-b-c-d, torsion為Vab和Vdc之夾角

In [15]:
def df_torsion(df):
    vxyz = ("VX", "VY", "VZ")
    vab = -df.loc[2:(len(df)-2), vxyz].values
    vdc = df.loc[4:len(df), vxyz].values
    vbc = -df.loc[3:(len(df)-1), vxyz].values
    lvbc = -df.loc[3:(len(df)-1), "bond_longth"].values
    vqb = vab - tothree(df_dot(vab,vbc)/(lvbc**2))*vbc
    vpc = vdc - tothree(df_dot(vdc,vbc)/(lvbc**2))*vbc
    lvqb = np.linalg.norm(vqb,axis=1)
    lvpc = np.linalg.norm(vpc,axis=1)
    phipsi = angle(vqb, vpc)
    #np.arccos(df_dot(vqb, vpc)/np.multiply(lvqb, lvpc))
    df["torsion"] = np.NaN
    df.loc[1:len(df)-3, "torsion"] = phipsi

In [16]:
def torsion_new(array):
    q1,q2,q3=array[0:3] #指派1-3行為q1q2q3
    q1Xq2 = np.cross(q1, q2) #第一個平面之法向量
    q2Xq3 = np.cross(q2, q3) #第二個平面之法向量
    n1 = q1Xq2/longth(q1Xq2) #q1Xq2單位向量
    n2 = q2Xq3/longth(q1Xq2) #q2Xq3單位向量
    u1 = n2 #第二個平面之單位法向量
    u3 = q2/longth(q2) #q2單位向量
    u2 = np.cross(u1, u3) 
    cos_theta = np.dot(n1, u1)
    sin_theta = np.dot(n1, u2)
    theta = -np.arctan2(sin_theta,cos_theta) 
    return theta

# 分chain並進行運算

輸入原始dataframe -> 分成數個chain -> reindex後分析各項數據 -> 把chain接回去並return

In [17]:
def analyze(df):    
    chain_group = df.groupby("chain") #依照chain分成數個groupby
    chain_index = chain_group.size().index # [A, B, C]
    df_dir = {} #創建一個字典存放abc chain
    
    for i in chain_index:
        df_dir[i] = chain_group.get_group(i) # {"A"= chainA之df}
        df_dir[i].index = range(1, len(df_dir[i])+1) # 分別對每個chain進行reindex 方便後續處理
        operation(df_dir[i])
        
    return pd.concat(df_dir)

產生向量、鍵長、鍵角、兩面角

In [18]:
def operation(df):
    df_vecter(df)
    df_bond_longth(df)
    df_bond_angle(df)
    vxyz = ("VX", "VY", "VZ")
    for i in range(2,len(df)-1):
        array = df.loc[i:i+2, vxyz].values
        df.loc[i+2:i+2,"torsion"] = torsion_new(array)

# 主程式
1.pdb -> df
2.分chain
3.進行運算
4.合併

In [ ]:
pdb = r'C:\Users\z9875\code\skp.pdb'
df = pdb_to_df(pdb)

final_df = analyze(df)

C:\Users\z9875\Anaconda3\lib\site-packages\pandas\core\frame.py:3140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]
C:\Users\z9875\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\z9875\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  
C:\Users\z9875\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 


In [ ]:
final_df

# 回推主鍊結構 
假設主練結構為 a-b-c-d 

In [ ]:
def pushback(df):#更改過
    df["AnsX"], df["AnsY"], df["AnsZ"] = np.NaN, np.NaN, np.NaN #將答案三欄先設為空值
    df.loc[1:3, ("AnsX", "AnsY", "AnsZ")] = df.loc[1:3, ("X", "Y", "Z")].values 
    for i in range(4, len(df)+1):
        vba, vcb =  df.loc[i-2:i-1, ("VX", "VY", "VZ")].values #向量 vba, vcb
        axis1 = np.cross(-vba, vcb) # 第一個旋轉軸是 vab和Vcb的法向量
        anglebca = df.loc[i-1, "bond_angle"] # 角bca
        v_tmp = Quaternion(axis=axis1, angle=anglebca).rotate(-vcb) #向量Vbc以axis1為軸旋轉angle
        angle2 = df.loc[i, "torsion"]
        v_tmp2 = Quaternion(axis=vcb, angle=-angle2).rotate(v_tmp) # V_tmp以vcb為軸旋轉angle2
        vdc = v_tmp2 * (df.loc[i, "bond_longth"]/df.loc[i-1, "bond_longth"]) # v_tmp2乘以 vbc長/vcd長
        df.loc[i:i, ("AnsX", "AnsY", "AnsZ")] =  df.loc[i-1:i-1, ("X", "Y", "Z")].values + vdc
    #df["error"] = np.sqrt(np.sum((df.loc[:, ("X","Y","Z")].values - df.loc[:, ("AnsX","AnsY","AnsZ")].values)**2, axis=1))
    df["error"] = df_longth(df.loc[:, ("X","Y","Z")].values - df.loc[:, ("AnsX","AnsY","AnsZ")].values)
    return df

In [ ]:
final_df.head()

In [ ]:
def pushback2(df):
    df["AnsX"], df["AnsY"], df["AnsZ"] = np.NaN, np.NaN, np.NaN #將答案三欄先設為空值
    df.loc[1:3, ("AnsX", "AnsY", "AnsZ")] = df.loc[1:3, ("X", "Y", "Z")].values 
    vxyz = ("VX", "VY", "VZ")
    vba, vcb =  df.loc[2:len(df)-2, vxyz].values, df.loc[3:len(df)-3, vxyz].values

### 回推和產生答案Dataframe

In [ ]:
ans_dir = {}
for i in chain_index:
    pushback(df_dir[i]) 
    ans_dir[i] = df_dir[i].loc[:, ("backbone_index","element", "X", "Y", "Z", "AnsX", "AnsY", "AnsZ", "error")]
ans = pd.concat(ans_dir)
#ans.loc[:, ("AnsX", "AnsY", "AnsZ")] = ans.loc[:, ("AnsX", "AnsY", "AnsZ")].round(decimals=3)

In [ ]:
ans.head()

In [ ]:
final_df.head()

In [ ]:
x = np.linspace(1, len(ans), len(ans))
y = ans.error.values

plt.scatter(x, y)

plt.xlabel("Atom number")
plt.ylabel("error (Å)")
plt.title("Unsorted values")

計算總誤差

In [61]:
sum(ans.error)

228.3493999311472